In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

In [2]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [3]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [4]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data_seq/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [6]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))

Reading lines...
Read 135842 sentence pairs
Trimmed to 135842 sentence pairs
Counting words...
Counted words:
fra 21334
eng 13043
['c etait il y a des annees .', 'it was ages ago .']


In [7]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [8]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input):
        hidden = self.initHidden()
        embedded = self.embedding(input).view(input.shape[0], 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size)

In [9]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size)

In [10]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size)
decoder1 = DecoderRNN(hidden_size, output_lang.n_words)

In [11]:
criterion = nn.NLLLoss()

In [13]:
# single training instance....
encoder_inp, target = tensorsFromPair(random.choice(pairs)) # get input and target pair....

enc_optimizer = optim.Adam(encoder1.parameters(),lr=0.0001)
dec_optimizer = optim.Adam(decoder1.parameters(),lr=0.0001)

output,hidden = encoder1(encoder_inp)

decoder_hidden = hidden

decoder_inp = torch.tensor([SOS_token])

loss = 0

for i in range(target.shape[0]):
    target_tensor = target[i].view(1)
    decoder_output, decoder_hidden = decoder1(decoder_inp, decoder_hidden)
    loss += criterion(decoder_output, target_tensor)
    decoder_inp = target_tensor
    
enc_optimizer.zero_grad()
dec_optimizer.zero_grad()
    
loss.backward()
print(f'loss: {loss}')    
enc_optimizer.step()
dec_optimizer.step()

loss: 66.72592163085938


Write a train function to train the encoder-decoder mode. You can modify the code snippet written above

In [15]:
def train(enc_model, dec_model, epochs=1, learning_rate=0.0001):
    enc_optimizer = optim.Adam(encoder1.parameters(),lr=learning_rate)
    dec_optimizer = optim.Adam(decoder1.parameters(),lr=learning_rate)
    
    for _ in range(epochs):
        for num in range(10000):
            
            encoder_inp, target = tensorsFromPair(random.choice(pairs))
            output,hidden = enc_model(encoder_inp)
            decoder_hidden = hidden
            decoder_inp = torch.tensor([SOS_token])
            loss = 0
            for i in range(target.shape[0]):
                target_tensor = target[i].view(1)
                decoder_output, decoder_hidden = dec_model(decoder_inp, decoder_hidden)
                loss += criterion(decoder_output, target_tensor)
                decoder_inp = target_tensor

            enc_optimizer.zero_grad()
            dec_optimizer.zero_grad()
    
            loss.backward()
    
            enc_optimizer.step()
            dec_optimizer.step()    

In [ ]:
train(encoder1, decoder1)

Write a function which which takes a trained model and generates a sentence translated in the target laguage. 

In [31]:
def generate(enc_model, dec_model, inp, output_lang, max_length=10):
    with torch.no_grad():
        output,hidden = enc_model(inp)
        decoder_hidden = hidden
        decoder_inp = torch.tensor([SOS_token])
        words = []
        l = 1
        while l<max_length:
            decoder_output, decoder_hidden = dec_model(decoder_inp, decoder_hidden)
            val = decoder_output.argmax(dim=1)
            word = output_lang.index2word[val.item()]
            if val.item()==1:
                break
            words.append(word)    
            decoder_inp = val
            l+=1
    return ' '.join(words)        
        
        

In [40]:
inp_sent, out_sent = random.choice(pairs)
inp, target = tensorsFromPair((inp_sent, out_sent))
generate(encoder1, decoder1, inp, output_lang)

'we re not .'

Modify the decoder class to incorporate attention mechanism. Consider dot-product of query and value vectors to calculate attention scores

In [81]:
class AttentionDecoderRNN(nn.Module):
    def __init__(self, hidden_size, enc_hidden_size, output_size):
        super(AttentionDecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(enc_hidden_size + hidden_size, output_size) # we will concatenate attention vector and decoder hidden
        self.softmax = nn.LogSoftmax(dim=1)
        self.attnsmax = nn.Softmax(dim=0)

    def forward(self, input, all_hidden, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        attn_vec = self.attnsmax(all_hidden.squeeze(1) @ hidden.squeeze(0).view(-1,1))*all_hidden.squeeze(1)
        attn_vec = torch.sum(attn_vec,0).unsqueeze(0)
        output = self.softmax(self.out(torch.cat((attn_vec,output[0]),1)))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size)

Write a function to calculate the BLEU score given a output translation and a reference translation

In [109]:
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
import math

In [112]:
def BLEU(out_trans, ref_trans):
    b_score = 0
    for n in range(4):
        out_grams = list(ngrams(word_tokenize(out_trans),n))
        ref_grams = list(ngrams(word_tokenize(ref_trans),n))

        ref_grams_cnt = Counter()
        for gram in ref_grams:
            ref_grams_cnt[gram]+=1

        score = 0
        for gram in set(out_grams):
            if gram in ref_grams_cnt:
                score+=ref_grams_cnt[gram]
        b_score+= score/len(out_grams)
    
    b_score/=4
    
    c = len(out_grams)
    r = len(ref_grams)
    b_p = 1 if c>r else math.exp(1-r/c)
    
    return b_p*b_score
    

In [113]:
BLEU('the cat the cat on the mat','the cat is on the mat')

0.5321428571428571